In [1]:
!pip install -q langchain langchain-community sentence-transformers faiss-cpu transformers PyPDF2 unstructured pdfminer.six pillow-heif pymupdf


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 24.6 MB/s eta 0:00:0000:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 60.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 62.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 98.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 74.5 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 82.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import os
import logging
from typing import List
from IPython.display import display
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain_community.llms import HuggingFacePipeline

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline


2025-08-06 04:00:07.579454: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754452807.774273      36 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754452807.834923      36 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [9]:
from IPython.display import display
import os

pdf_path = "/kaggle/input/fitness/sample_fitness.pdf"

if not os.path.exists(pdf_path):
    print("❌ PDF not found. Please upload it manually using the Jupyter file browser.")
else:
    print("✅ PDF found!")


✅ PDF found!


In [10]:
# def load_and_chunk(file_path: str, chunk_size: int = 500, chunk_overlap: int = 100):
#     from langchain.text_splitter import RecursiveCharacterTextSplitter

#     loader = PyPDFLoader(file_path)
#     documents = loader.load()

#     splitter = RecursiveCharacterTextSplitter(
#         chunk_size=chunk_size,
#         chunk_overlap=chunk_overlap
#     )

#     chunks = splitter.split_documents(documents)
#     return chunks

from langchain_community.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

def load_and_chunk(file_path):
    loader = PyMuPDFLoader(file_path)
    documents = loader.load()

    splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
    return splitter.split_documents(documents)


docs = load_and_chunk(pdf_path)
print(f"✅ Loaded {len(docs)} chunks.")


✅ Loaded 4 chunks.


In [11]:
def embed_and_store(docs, index_name="index_store"):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    db = FAISS.from_documents(docs, embeddings)
    db.save_local(index_name)
    return db

db = embed_and_store(docs)


In [12]:
def load_retriever(index_name="index_store"):
    embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
    return FAISS.load_local(
        folder_path=index_name,
        embeddings=embeddings,
        allow_dangerous_deserialization=True  # ✅ This enables safe loading
    ).as_retriever()
retriever = load_retriever()

In [13]:
from transformers import AutoModelForCausalLM, AutoTokenizer
model_name = "Qwen/Qwen3-0.6B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

llm_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=128,
    do_sample=True,
    temperature=0.2,
    device=0  # CPU
)

llm = HuggingFacePipeline(pipeline=llm_pipeline)


Device set to use cuda:0


In [14]:
rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)
print("✅ RAG pipeline ready.")


✅ RAG pipeline ready.


In [15]:
while True:
    query = input("\nYou: ")
    if query.lower() in ['exit', 'quit']:
        break

    response = rag_chain.invoke({"query": query})
    print("🤖:", response["result"])



You:  What's the workout plan?


🤖: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

Fat Loss Workout and Indian Vegetarian Diet Plan
Height: 5'6" | Weight: 170 lbs | Target: 150 lbs
Workout: 5 Days/Week | Rest Days: 2/Week
Preference: Minimal chicken, no fish, mostly Indian vegetarian meals
Goal: Fat loss with strength training and a high-protein Indian diet
Workout Plan (Weekly Breakdown)
Day 1 – Push: Bench press, shoulder press, lateral raises, triceps pushdowns, cardio
Day 2 – Pull: Lat pulldown, rows, curls, face pulls, planks, leg raises

Day 2 – Pull: Lat pulldown, rows, curls, face pulls, planks, leg raises
Day 3 – Legs: Squats, lunges, deadlifts, leg curl, calf raises, cardio
Day 4 – Rest: Light activity or complete rest
Day 5 – Conditioning: Kettlebell swings, ropes, thrusters, sled/row, jumps, woodchoppers
Day 6 – Hybrid: Incline press, cable row, curls, dips, cardio
Day 7 – Rest: Full rest an


You:  exit
